# INST326 — Week 3 Lecture: Functions in Python
*Date:* September 22, 2025

---

### Learning Objectives
By the end of this lecture, you should be able to:
- Define and call functions using parameters and return values.
- Use positional, keyword, default, and variable-length arguments (`*args`, `**kwargs`) appropriately.
- Explain scope (local vs. global) and side effects; prefer pure functions for testability.
- Write helpful docstrings and type hints for readability and tooling.
- Decompose problems into small, reusable functions within a simple module.
- Apply these practices to our **Garden Management System** demonstration project.

> **Context:** In Week 2 we used variables, conditionals, and loops. Week 3 adds **functions** so we can structure programs, reduce duplication, and prepare for object‑oriented design in upcoming weeks.


---
### Agenda
1. Why functions? (motivation & design heuristics)  
2. Function syntax and calling conventions  
3. Parameters & arguments: positional, keyword, defaults, `*args`, `**kwargs`  
4. Return values, multiple returns, early returns  
5. Scope, side effects, and purity  
6. Documentation & type hints (docstrings, `typing`)  
7. Garden Management mini‑module: utilities for seeds & beds  
8. Testing light: quick checks with `assert`  
9. **Your Turn** practice set (at the end)


---
### 1) Why Functions?
- Break large problems into smaller, testable parts
- Reuse logic across your program
- Name behavior to improve readability
- Prepare for later refactoring into classes and methods

> **Heuristic:** If you copy‑paste code or a block exceeds ~10–15 lines, consider a function.


---
### 2) Function Syntax & Calls
Basic pattern:
```python
def name(parameters):
    """One‑line summary; optional details and parameter docs."""
    # body
    return result  # optional
```


**Explanation:** Functions in Python always start with the keyword `def`. They can take parameters inside parentheses, and the body is indented. A `return` statement sends a value back to the caller. If you omit `return`, the function gives back `None` by default.

In [ ]:
# Minimal example

def square(n):
    """Return n squared."""
    return n * n

print(square(4))


---
### 3) Parameters & Argument Styles
- **Positional**: order matters
- **Keyword**: name=value at call site
- **Defaults**: provide a fallback value
- **Variable‑length**: `*args` (tuple), `**kwargs` (dict)

> Prefer keyword args when there are 3+ parameters or when meaning could be ambiguous.


**Explanation:** Parameters are variables defined in the function signature. Arguments are the actual values you pass when calling the function. Default values make parameters optional, while `*args` and `**kwargs` let you handle flexible numbers of inputs.

In [4]:
def format_plant(name, species, quantity=1, urgent=False):
    """Return a formatted line for a plant order."""
    tag = "URGENT" if urgent else "normal"
    return f"{name} ({species}) x{quantity} [{tag}]"

# Positional
print(format_plant("Sun Gold", "Tomato", 3, True))
# Keyword for clarity
print(format_plant(name="Genovese Basil", species="Basil", urgent=False))

T = (20, 'Sean', 35, 75, [30, 60, 90])
print(T[4])

Sun Gold (Tomato) x3 [URGENT]
Genovese Basil (Basil) x1 [normal]
[30, 60, 90]


In [5]:
def demo_varargs(first, *extras, **options):
    """Show how *args and **kwargs are captured."""
    print("first:", first)
    print("extras:", extras)
    print("options:", options)

demo_varargs("bed-1", "row-A", "row-B", "row-C", mulch=True, spacing_cm=30)


first: bed-1
extras: ('row-A', 'row-B', 'row-C')
options: {'mulch': True, 'spacing_cm': 30}


---
### 4) Return Values & Early Returns
- A function may return **one value** or a **tuple** of values
- Use **early returns** to improve readability


**Explanation:** Functions can return a single value or a tuple of values. Using multiple return values helps when you want to provide additional information. Early returns make code easier to read by exiting as soon as a condition is met.

In [2]:
def fertilizer_needed(nitrogen_ppm, target_ppm=50):
    """Return whether fertilizer is needed and by how much.
    Returns (needed: bool, deficit: float).
    """
    if nitrogen_ppm >= target_ppm:
        return (False, 0.0)  # early return
    deficit = target_ppm - nitrogen_ppm
    return (True, deficit)

print(fertilizer_needed(45))
print(fertilizer_needed(60))


(True, 5)
(False, 0.0)


---
### 5) Scope, Side Effects, and Purity

**Scope**
- Variables created inside a function are **local**.
- Avoid modifying global state from inside functions (harder to reason about).

**Purity**
- A *pure* function's output depends only on inputs and has no side effects.
- Pure functions are easier to test and reuse.

> Pragmatic rule: prefer pure functions for computations; isolate I/O (prints, file writes) in thin wrappers.


**Explanation:** Variables created inside a function are local and disappear when the function ends. If you want to use a global variable, you must declare it, but this is discouraged. Pure functions avoid modifying state outside themselves, which makes them predictable and easier to test.

In [ ]:
total_applied = 0  # global state (we'll avoid mutating it inside compute)

def compute_garden_area(width_m, length_m):
    """Pure function: computes area in square meters."""
    return width_m * length_m

print("Area:", compute_garden_area(2.0, 5.5))


---
### 6) Docstrings & Type Hints
- **Docstrings** explain *what* and *why* (not just how)
- **Type hints** help tools and readers; Python doesn't enforce at runtime by default


**Explanation:** A docstring is a string at the beginning of a function that explains what the function does. Tools like `help()` or IDEs display docstrings. Type hints improve readability and help catch mistakes during development, but Python does not enforce them at runtime.

In [12]:
from typing import List, Dict, Tuple

def plants_per_bed(bed_len_m: str, spacing_cm: int) -> int:
    """Estimate count of plants that fit in a single row.

    Parameters
    ----------
    bed_len_m : float
        Length of the bed in meters.
    spacing_cm : float
        Spacing between plants in centimeters.

    Returns
    -------
    int
        Estimated number of plants.
    """
    spacing_m = spacing_cm / 100
    if spacing_m <= 0:
        raise ValueError("spacing must be positive")
    return int(bed_len_m / spacing_m)

print(plants_per_bed(4.8, 30))
print(7 / 2)
print(7 // 2)

16
3.5
3


---
### 7) Garden Management Mini‑Module (Functions)
We'll write a tiny utility module *inline* for now; later you'll move functions to a `.py` file.

**Goal:** Calculate seed start schedules and labels.


In [15]:
from datetime import date, timedelta

FROST_DATE = date(2026, 4, 15)  # example for College Park, MD (approximate)

def weeks_before_last_frost(crop: str) -> int:
    """Return recommended weeks to start indoors before last frost.
    Values are simplified for teaching.
    """
    table = {
        "tomato": 6,
        "pepper": 8,
        "basil": 4,
        "eggplant": 8,
        "cucumber": 3,
    }
    return table.get(crop.lower(), 4)


def seed_start_date(crop: str, frost_date: date = FROST_DATE) -> date:
    weeks = weeks_before_last_frost(crop)
    return frost_date - timedelta(weeks=weeks)


def make_label(crop: str, variety: str, start: date) -> str:
    return f"{crop.title()} — {variety} | Start: {start.isoformat()}"

# Demo
for crop, variety in [("tomato", "Sun Gold"), ("basil", "Genovese"), ("pepper", "Jalapeño")]:
    start = seed_start_date(crop)
    print(make_label(crop, variety, start))

assert 7 / 3 == (2 + 1/3)

Tomato — Sun Gold | Start: 2026-03-04
Basil — Genovese | Start: 2026-03-18
Pepper — Jalapeño | Start: 2026-02-18


---
### 8) Quick Checks with `assert` (a small preview of code testing)
Small, focused assertions can catch regressions early.


**Explanation:** Assertions are a lightweight way to confirm that a function behaves as expected. If an assertion fails, Python raises an error immediately, which helps catch mistakes early in development.

In [ ]:
# Quick sanity tests
assert plants_per_bed(4.0, 40) == 10
assert weeks_before_last_frost("Tomato") == 6

# Edge case check for ValueError
try:
    # plants_per_bed(1.0, 0)
except ValueError:
    print("Caught expected ValueError for spacing=0")


# Creating Your First Module: `garden_utils.py`

In this notebook, you will learn how to:
1. Write Python functions in a separate `.py` file (a **module**).
2. Save the file so it can be imported into your notebook or other programs.
3. Use `import` to call your functions from the module.

This is the foundation of the **GitHub Utility Library** assignment.


---
## Step 1: Why Create a Module?
So far, we've written functions inside Jupyter notebooks. But in real projects:
- Code should be reusable across files and projects.
- Modules (`.py` files) allow us to **organize functions** neatly.
- This makes it easier to test, maintain, and share code.


---
## Step 2: Create `garden_utils.py`

Open your editor (VS Code or Colab file explorer) and create a new file named:

```
garden_utils.py
```

Paste the following starter code into the file:


In [ ]:
"""
garden_utils.py
Utility functions for the Garden Management System (Week 3).
Students can expand this module with more functions as the semester progresses.
"""

from datetime import date, timedelta

# Example frost date (College Park, MD). Update as needed.
FROST_DATE = date(2026, 4, 15)

def weeks_before_last_frost(crop: str) -> int:
    """Return recommended weeks to start indoors before last frost."""
    table = {
        "tomato": 6,
        "pepper": 8,
        "basil": 4,
        "eggplant": 8,
        "cucumber": 3,
    }
    return table.get(crop.lower(), 4)

def seed_start_date(crop: str, frost_date: date = FROST_DATE) -> date:
    """Calculate recommended seed start date based on crop and frost date."""
    weeks = weeks_before_last_frost(crop)
    return frost_date - timedelta(weeks=weeks)

def make_label(crop: str, variety: str, start: date) -> str:
    """Return a formatted label for seed starting trays."""
    return f"{crop.title()} — {variety} | Start: {start.isoformat()}"


---
## Step 3: Save the File

Save the file as **`garden_utils.py`** in the same folder as this notebook.

If you're using Google Colab:
- Click the file icon (left panel).
- Right-click → *New File* → name it `garden_utils.py`.
- Paste the code above, then save.

If you're using VS Code:
- Right-click the folder → *New File* → name it `garden_utils.py`.
- Paste the code above and save.


---
## Step 4: Import and Use the Module

Now that `garden_utils.py` exists in the same folder, we can **import it** and use the functions.


In [ ]:
import garden_utils

start = garden_utils.seed_start_date("tomato")
print(garden_utils.make_label("tomato", "Sun Gold", start))

---
## Step 5: Extend Your Module

Try adding new functions to `garden_utils.py`, such as:

- `cm_to_m(cm)` – convert centimeters to meters.

- `plants_per_bed(bed_len_m, spacing_cm)` – estimate how many plants fit in a row.

- `rows_that_fit(bed_width_m, row_spacing_cm)` – compute how many rows fit across the bed.


Each time you edit `garden_utils.py`, save the file and re-import it in the notebook.
To reload without restarting the kernel, you can use:


In [ ]:
import importlib
importlib.reload(garden_utils)

---
## Your Turn (Practice)
Work these on your own or with a partner. Use functions. Keep each function short and testable.

1. **Refactor Repetition:** Write a function `cm_to_m(cm)` and use it to refactor any calculations using centimeters to meters in previous examples.

2. **Row Planner:** Write `rows_that_fit(bed_width_m, row_spacing_cm)` that returns how many rows fit in a bed.

3. **Bed Planner:** Combine `plants_per_bed` and `rows_that_fit` to write `plants_per_rectangle(width_m, length_m, spacing_cm, row_spacing_cm)`.

4. **Safer Label:** Extend `make_label` to include a `sown` boolean keyword argument (default `False`) and return a label like `"Tomato — Sun Gold | Start: 2026-03-04 | SOWN: no"`.

5. **Flexible Start:** Modify `weeks_before_last_frost` to accept an optional `override` dict (`**kwargs`) that can temporarily override the table for a specific crop when calling the function.

6. **Validation:** Update one function to raise a `ValueError` for invalid inputs (e.g., negative spacing). Add a few `assert` tests.

7. **Docstrings & Types:** Add docstrings and type hints to all your functions. Use meaningful parameter names.

8. **Mini‑Dataset Function:** Given a list of `(crop, variety, spacing_cm)` tuples, write a function that returns a list of formatted labels with computed start dates.

9. **Early Return:** Write `needs_starting_indoors(crop)` that returns `False` early for crops typically direct‑sown (e.g., carrots), otherwise `True`.

10. **Stretch (Optional):** Write a small `main()` function that calls your planner functions for two crops and prints a short plan. (We’ll formalize `if __name__ == "__main__":` later.)


---
### Wrap‑Up
- Functions help us structure programs and prepare for OOP.
- Aim for small, pure functions with clear names and docstrings.
- Use keyword arguments and sensible defaults.
- Add quick `assert` checks while you code.

**Next Week Preview:** Modules, packaging basics, and beginning to connect functions into larger program structure.
